
# relicot: CoT & CoT-UQ Quick Tests

このノートブックは、**CoT モジュール**と**CoT-UQ モジュール**の動作確認を最小構成で行います。  
実行は **リポジトリのルート** で行ってください（`relicot/` パッケージをインポートできるように）。

> 依存関係：本ノートは **ダミー生成関数／ダミースコアラ**で動作します。  
> HuggingFace の実モデルで試す場合は、後半のセルのコメントを外して使用してください。


In [ ]:

import os, sys
# リポジトリ直下で実行している想定（例: ./relicot/CoT/ ...）
if os.path.isdir('relicot') and os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())
print('sys.path[0]:', sys.path[0])


## 1. ダミー生成関数（モデル不要）で CoT をテスト

In [ ]:

from relicot.CoT.cot_module import CoTGenerator
from relicot.CoT.cot_eval import EvalItem, evaluate_single, evaluate_self_consistency, vote_share
from relicot.CoT.cot_eval import confidence_from_vote_share, expected_calibration_error, brier_score

QUESTION = "犬は色を識別できますか？"

def dummy_generate(prompt: str, max_new_tokens=256, temperature=0.7, top_p=0.95, stop=None):
    # JSONモード検出
    if "JSONで出力してください" in prompt or '{"steps":' in prompt:
        return '{"steps": ["犬の網膜には錐体があり色覚は限定的", "青と黄の弁別が中心"], "answer": "はい、限定的に可能"}'
    # キーワード抽出プロンプト検出（UQ用）
    if "Keywords for Each Reasoning Step" in prompt:
        return "Step 1: 錐体細胞 (/8/); 青黄感受性 (/7/)\nStep 2: NO ANSWER\nStep 3: 二色型視覚 (/9/)"
    # 通常CoT
    return (
        "Step 1: 犬の視細胞は杆体優位だが錐体も存在する。\n"
        "Step 2: 錐体の種類が限られ、青と黄の弁別は可能だが赤は弱い。\n"
        "Answer: はい、限定的に可能"
    )

gen = CoTGenerator(generate_fn=dummy_generate)

# run()
out = gen.run(problem=QUESTION, template="cot_default_ja")
print("CoT answer:", out["answer"])
print("CoT steps:", out["steps"])

# run_json()
out_json = gen.run_json(problem=QUESTION, template="cot_json_ja")
print("JSON answer:", out_json["answer"])

# run_self_consistency()
sc = gen.run_self_consistency(problem=QUESTION, template="cot_sc_ja", k=5)
print("SC answers:", sc["answers"])
print("SC vote share:", vote_share(sc["answers"]))


## 2. 評価（正解判定・自己一貫性・キャリブレーションの素振り）

In [ ]:

# 正解判定（文字列正規化一致）
item = EvalItem(problem=QUESTION, gold="はい、限定的に可能")
res = evaluate_single(out["answer"], out["steps"], item)
print("correct:", res.correct, "reason:", res.reason, "step_stats:", res.step_stats)

# SC の最大得票率 → 確信度 proxy
share = vote_share(sc["answers"])
conf_sc = confidence_from_vote_share(share)
print("SC vote max:", conf_sc)

# 簡易ECE/Brier（ダミー：SCのmaxをconfとして使う例）
ece = expected_calibration_error([conf_sc], [True])
brier = brier_score(conf_sc, True)
print("ECE (1-bin demo):", ece, "Brier:", brier)


## 3. CoT-UQ（ダミースコアラでAP/SEを試す）

In [ ]:

from relicot.UQ.uq_module import run_cotuq

# ダミースコアラ（HuggingFace不要）
class DummyScorer:
    def __init__(self): pass
    def seq_logprob(self, prompt: str, continuation: str):
        # continuation（キーワード）が短いほど確率が高いという適当な近似
        import math
        n = max(1, len(continuation))
        avg_lp = - (n / 20)  # 長いと低スコア
        return type("R", (), {"total_logprob": avg_lp * n, "avg_logprob": avg_lp, "n_tokens": n})()
    def binary_choice_prob(self, prompt: str, opt_true: str=" True", opt_false: str=" False") -> float:
        return 0.72  # 固定で True 72%

scorer = DummyScorer()

out_uq = run_cotuq(gen, QUESTION, scorer=scorer)
print("AP confidence:", out_uq.ap_confidence)
print("SE confidences:", out_uq.se_confidences)
print("KW by step:", out_uq.kw_by_step)


## 4. 確信度の合成（SC × UQ(AP/SE)）

In [ ]:

from relicot.CoT.cot_eval import ConfidenceSignals, ConfidencePolicy, combine_confidence

share = vote_share(sc["answers"])
ap = out_uq.ap_confidence or 0.0
se = max(out_uq.se_confidences.values()) if out_uq.se_confidences else 0.0
sig = ConfidenceSignals(sc_vote_share_max=(max(share.values()) if share else None),
                        ap_conf=ap, se_conf=se)
final_conf = combine_confidence(sig, ConfidencePolicy())
print("Final combined confidence:", final_conf)


## 5. （任意）HuggingFace の小モデルでの実行例

In [ ]:

# ▼ 実モデルで試す場合はコメント解除
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch
# from relicot.UQ.uq_module import HFCausalLogProbScorer
#
# model_id = "sshleifer/tiny-gpt2"
# tok = AutoTokenizer.from_pretrained(model_id)
# mdl = AutoModelForCausalLM.from_pretrained(model_id).eval()
# if torch.cuda.is_available():
#     mdl = mdl.to("cuda")
#
# gen_hf = CoTGenerator(
#     generate_fn=lambda p, **kw: tok.decode(mdl.generate(
#         **tok(p, return_tensors="pt").to(mdl.device), max_new_tokens=128, do_sample=True
#     )[0], skip_special_tokens=True)
# )
# scorer_hf = HFCausalLogProbScorer(mdl, tok)
# out_hf = run_cotuq(gen_hf, QUESTION, scorer=scorer_hf)
# out_hf.ap_confidence, out_hf.se_confidences
